In [ ]:
# importing the required modules  
import json
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaProducer


In [ ]:
from faker import Faker
import random
from faker.providers import BaseProvider
from datetime import datetime
import time

In [ ]:
# Kafka producer configuration
producer = KafkaProducer(bootstrap_servers=['0.0.0.0:9092'],
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))


In [ ]:
fake = Faker()

# Create required fields with values
class InstrumentProvider(BaseProvider):
    def instr_name(self):
        instr_names = ["RIL", "INFY", "ONGC", "TCS"]
        return instr_names[random.randint(0,len(instr_names)-1)]
    
    def trxn_type(self):
        trxn_type = ["BUY", "SELL"]
        return trxn_type[random.randint(0,len(trxn_type)-1)]
    
    def exchange(self):
        trxn_type = ["NSE", "BSE"]
        return trxn_type[random.randint(0,len(trxn_type)-1)]

fake.add_provider(InstrumentProvider)

In [ ]:
# creating function to generate the Order
def produce_order(orderid=1):
    instrument = fake.instr_name()
    trxn_type = fake.trxn_type()
    quantity = round(random.uniform(2, 5), 0)
    price = round(random.uniform(10, 20), 0)
    order_ts = datetime.now().strftime("%Y/%m/%d, %H:%M:%S")
    
    # message
    message = {
        'id': orderid+1000000,
        'instrument': instrument,
        'trxn_type': trxn_type,
        'quantity': quantity,
        'price': price,
        'order_timestamp': order_ts,
    }
    return message

# produce_order(8)


In [ ]:
# Produce buy/sell orders
i=1
while i < 1200:
    message = produce_order(i)

    print("Sending: {}".format(message))
    # sending the message to Kafka
    producer.send("stock_exchange_orders",
                  value=message)
    # 1 seconds of sleep time before the next message
    time.sleep(0.5)

    # Force sending of all messages
    if (i % 100) == 0:
        producer.flush()
    i=i+1
producer.flush()